# CodeTrans-T5-Small-ST-TF-Python

This notebook explores the CodeTrans model which is based on the T5 model's architecture. This particular model uses the T5-Small skeleton and is built for the purpose of a single task (ST), to generate code documentation for Python. This notebook will first use the model from HuggingFace and then attempt to fine-tune it (FT) on the [`code docstring corpus`](https://github.com/EdinburghNLP/code-docstring-corpus).

## References
1. [CodeTrans on GitHub](https://github.com/agemagician/CodeTrans)
2. [Models' HuggingFace page](https://huggingface.co/SEBIS/code_trans_t5_small_code_documentation_generation_python?text=def+log_directory_structure%28directory_path%2C+ai_context%2C+indent%3D0%29%3A%0D%0A++++if+not+os.path.exists%28directory_path%29%3A%0D%0A++++++++print%28%22Directory+not+found.%22%29%0D%0A++++++++return%0D%0A%0D%0A++++%23+Get+the+list+of+items+in+the+directory%0D%0A++++items+%3D+os.listdir%28directory_path%29%0D%0A%0D%0A++++for+item+in+items%3A%0D%0A++++++++item_path+%3D+os.path.join%28directory_path%2C+item%29%0D%0A%0D%0A++++++++%23+Check+if+the+item+is+a+directory%0D%0A++++++++if+os.path.isdir%28item_path%29%3A%0D%0A++++++++++++%23+Log+the+directory+using+ai_context%0D%0A++++++++++++directory_name+%3D+os.path.basename%28item_path%29%0D%0A++++++++++++indentation+%3D+%22++%22+*+indent++%23+Adjust+indentation+for+subdirectories%0D%0A++++++++++++log_message+%3D+f%22%7Bindentation%7DDirectory%3A+%7Bdirectory_name%7D%22%0D%0A++++++++++++print%28log_message%29%0D%0A%0D%0A++++++++++++%23+Recursively+log+the+subdirectory+structure%0D%0A++++++++++++log_directory_structure%28item_path%2C+ai_context%2C+indent+%2B+1%29%0D%0A%0D%0A++++++++%23+If+it%27s+a+file%2C+you+can+log+it+similarly)
3. [Dataset on GitHub](https://github.com/EdinburghNLP/code-docstring-corpus) 

---